In [ ]:
#file: forest_demo.ipynb
# Author Petri Lamminaho 

# Code build, train and test simple random forest algorithm
# Uses Python 3 

In [225]:
from random import seed, randrange
from csv import reader
from math import sqrt

In [226]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row: 
                continue
            dataset.append(row)
    return dataset

In [251]:
#load_csv('data.csv')

In [228]:
def str_column_to_flt(dataset, column):
    for row in dataset:
        row[column] = float( row[column].strip() )

In [229]:
def str_column_to_int(dataset, column):
    class_values = [ row[column] for row in dataset ]
    unique = set( class_values )
    lookup = dict()
    for i, value in enumerate( unique ): lookup[value] = i
    for row in dataset: row[column] = lookup[row[column]]
    return lookup

In [230]:
#str_column_to_int(dataset, 1)

In [231]:
# Split a dataset num of folds 
def cross_validation_split(dataset, n_folds):
    data_split = list()
    copyed_data = list(dataset)
    fold_size = int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(copyed_data))
            fold.append(copyed_data.pop(index))
        data_split.append(fold)
    return data_split

In [232]:
#cross_validation_split(dataset, 5)

In [233]:
# Calculate accuracy percentage 
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]: correct += 1
    return 100. * correct / float( len(actual) )


In [234]:
# Evaluate an algorithm using a cross validation split
def train_and_test_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [235]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, test_splitright = list(), list()
    for row in dataset:
        if row[index] < value: left.append(row)
        else: right.append(row)
    return left, right

In [236]:
# Calculate the Gini index for a split dataset
def get_gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0: continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

In [237]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features: features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = get_gini_index(groups, class_values)
            if gini < b_score: 
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}


In [238]:
# Create leaf node value
def create_leaf(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [239]:
# Create child splits for a node or make leaf
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = create_leaf(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = create_leaf(left), create_leaf(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = create_leaf(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = create_leaf(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [240]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(dataset, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root
 

In [241]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict): 
            return predict(node['left'], row)
        else: return node['left']
    else:
        if isinstance(node['right'], dict): 
            return predict(node['right'], row)
        else:
            return node['right'] 

In [242]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample 

In [243]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [244]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

In [252]:
# Test the random forest algorithm
seed(1)
# load and prepare data
dataset = load_csv('data.csv')
# convert string attributes to integers
#for i in range(0, len(dataset[0])-1):
#    str_column_to_flt(dataset, i)
# convert class column to integers
#str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
    scores = train_and_test_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('')

Trees: 1
Scores: [73.17073170731707, 80.48780487804878, 68.29268292682927, 56.09756097560975, 60.97560975609756]
Mean Accuracy: 67.805%

Trees: 5
Scores: [78.04878048780488, 70.73170731707317, 63.41463414634146, 70.73170731707317, 65.85365853658537]
Mean Accuracy: 69.756%

Trees: 10
Scores: [78.04878048780488, 65.85365853658537, 78.04878048780488, 70.73170731707317, 73.17073170731707]
Mean Accuracy: 73.171%

